In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

In [2]:
import requests, io

In [ ]:
url1 = "https://diaryappbackend-gmme.onrender.com/generate"
response1 = requests.get(url1)
print(response1.json())
request_list = response1.json()

In [4]:
base_model = "stabilityai/stable-diffusion-xl-base-1.0"

In [ ]:
pipe = StableDiffusionXLPipeline.from_pretrained(base_model, torch_dtype=torch.bfloat16)

In [6]:
pipe = pipe.to("cuda")

In [ ]:
for generate_request in request_list:
    prompt = generate_request['text_payload'] + " , paper sketch"

    filename = generate_request['imageName']

    image = pipe(prompt).images[0]

    image_bytes = io.BytesIO()
    image.save(image_bytes, format='JPEG')  # Explicitly save as JPEG
    image_bytes.seek(0)

    url2 = "https://diaryappbackend-gmme.onrender.com/image"
    files = {
        'image': (filename, image_bytes, 'image/jpeg')
    }
    data = {
        'filename': filename
    }
    
    response2 = requests.post(url2, files=files, data=data)
    print(response2.status_code, response2.text)

    if response2.status_code == 200 or response2.status_code == 201:
        url3 = "https://diaryappbackend-gmme.onrender.com/generate"

        payload = {
            "imageName": generate_request['imageName']
        }
        
        response3 = requests.delete(url3, json=payload)
        print(response3.status_code, response3.text)
    
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

In [8]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()